In [1]:
# !pip install catboost
# !pip install xgboost

# 1. Import Necessary Libraries 

In [38]:
# import necessary libraries
import pandas as pd #library for data manipulation
import numpy as np # library for working with arrays

# import preprocessing libraries

# creating visualizations in Python
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
import seaborn as sns
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# import modelling libraries
from catboost import CatBoostRegressor #features built in support for handling categorical features
from xgboost import XGBRegressor #designed to be fast and efficient: for both classification and regression
from sklearn.ensemble import RandomForestRegressor #creates multiple decision trees
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, RepeatedStratifiedKFold

# pd.options.display.max_rows=2000
pd.set_option('display.max_rows', 100) #sets maximum rows to 100
pd.set_option('display.max_columns', 30) #sets maximum columns to 30

# 2. Read Data

In [3]:
# read csv data
df = pd.read_csv('wfp_food_prices_ken.csv', parse_dates=True, index_col='date')

In [4]:
# check sample data/records
df.sample(5)

,admin1,admin2,market,latitude,longitude,category,commodity,unit,priceflag,pricetype,currency,price,usdprice
date,,,,,,,,,,,,,
2021-01-15,Rift Valley,Baringo,Marigat town (Baringo),0.469569,35.980663,cereals and tubers,"Maize (white, dry)",90 KG,actual,Wholesale,KES,2430.0,22.2121
2006-09-15,Nairobi,Nairobi,Nairobi,-1.283333,36.816667,pulses and nuts,Beans,KG,actual,Wholesale,KES,34.75,0.4783
2021-04-15,Coast,Mombasa,Moroto (Mombasa),-4.033844222,39.65619357,cereals and tubers,Potatoes (Irish),KG,aggregate,Retail,KES,56.18,0.531
2018-02-15,Nairobi,Nairobi,Nairobi,-1.283333,36.816667,cereals and tubers,Bread,400 G,actual,Retail,KES,45.0,0.4514
2021-01-15,Nairobi,Nairobi,Kitengela (Kajiado),-1.285515,36.8556,vegetables and fruits,Cowpea leaves,KG,actual,Wholesale,KES,30.0,0.2742


# 3. Data Type description and statistical information

In [5]:
# check shape of the dataset
df.shape

(14755, 13)

In [6]:
# check information of the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>

Index: 14755 entries, #date to 2022-08-15

Data columns (total 13 columns):

 #   Column     Non-Null Count  Dtype 

---  ------     --------------  ----- 

 0   admin1     14755 non-null  object

 1   admin2     14755 non-null  object

 2   market     14755 non-null  object

 3   latitude   14755 non-null  object

 4   longitude  14755 non-null  object

 5   category   14755 non-null  object

 6   commodity  14755 non-null  object

 7   unit       14755 non-null  object

 8   priceflag  14755 non-null  object

 9   pricetype  14755 non-null  object

 10  currency   14755 non-null  object

 11  price      14755 non-null  object

 12  usdprice   14755 non-null  object

dtypes: object(13)

memory usage: 1.6+ MB


### 
* From the above, all columns are categorical. Shows there are anomalies since some 
  columns cant have object as a datatype i.e price etc.
* No missing values from the dataset

In [7]:
# check statistical information of numerical data
# df.describe()

# 4. Data Preprocessing

In [8]:
# check for duplicates
df.duplicated().sum()

251

In [9]:
# remove duplicates
df = df.drop_duplicates()
df.duplicated().sum()

0

In [10]:
# check for missing values and sorting the issue
missing_perc = df.isnull().mean()*100
missing_values = pd.DataFrame({'column_name' : df.columns, 'Missing Percentage' : missing_perc})
missing_values

,column_name,Missing Percentage
admin1,admin1,0.0
admin2,admin2,0.0
market,market,0.0
latitude,latitude,0.0
longitude,longitude,0.0
category,category,0.0
commodity,commodity,0.0
unit,unit,0.0
priceflag,priceflag,0.0
pricetype,pricetype,0.0


### 
- No missing values from the dataset

In [11]:
# make a copy of the dataframe
df1 = df.copy()

In [12]:
# rename some columns
df1 = df1.rename(columns={'admin1': 'Province', 'admin2': 'County'})

In [13]:
# column names
print(*df1.columns, sep='\n')

Province

County

market

latitude

longitude

category

commodity

unit

priceflag

pricetype

currency

price

usdprice


In [14]:
# check datatypes of the above columns
for col in df1:
    print(f'{col} : {df1[col].dtypes}')

Province : object

County : object

market : object

latitude : object

longitude : object

category : object

commodity : object

unit : object

priceflag : object

pricetype : object

currency : object

price : object

usdprice : object


In [15]:
# remove some patterns in the cols you want to convert into numerical
num_cols = ['latitude', 'longitude', 'price', 'usdprice']
import re
def numeric(text):
    text = text.lower()
    if re.search(r'#', text):
        text = 0
    return text

df1[num_cols] = df1[num_cols].applymap(numeric)

In [16]:
# convert necessary columns to their respective datatypes
num_cols = ['latitude', 'longitude', 'price', 'usdprice']
for num in num_cols:
    df1[num] = df1[num].astype('float64')
    
df1.dtypes

Province      object
County        object
market        object
latitude     float64
longitude    float64
category      object
commodity     object
unit          object
priceflag     object
pricetype     object
currency      object
price        float64
usdprice     float64
dtype: object

### 
- I now have a clean dataset with their corresponding data types

In [17]:
# check statistical information of numerical data
df1.describe()

,latitude,longitude,price,usdprice
count,14504.000000,14504.000000,14504.000000,14504.000000
mean,-0.215473,36.901888,1934.812354,18.891459
std,2.105010,1.840700,2865.725311,26.884711
min,-4.659730,0.000000,0.000000,0.000000
25%,-1.283333,35.283333,53.360000,0.553675
50%,-0.303099,36.816667,184.120000,1.820700
75%,0.516667,37.989172,3000.000000,31.058300
max,3.936804,41.858383,19800.000000,183.806900


### 
- No outliers in my dataset

### 4.1 Quality mapping

In [18]:
# columns
df1.columns

Index(['Province', 'County', 'market', 'latitude', 'longitude', 'category',
       'commodity', 'unit', 'priceflag', 'pricetype', 'currency', 'price',
       'usdprice'],
      dtype='object')

In [19]:
# check category 
print(*df1['category'].unique(), sep='\n')

#item+type

cereals and tubers

pulses and nuts

milk and dairy

oil and fats

non-food

meat, fish and eggs

miscellaneous food

vegetables and fruits


### 
- category seems to be ok

In [20]:
# check commodity 
print(*df1['commodity'].unique(), sep='\n')

#item+name

Maize

Maize (white)

Beans

Beans (dry)

Potatoes (Irish)

Sorghum

Bread

Milk (cow, pasteurized)

Oil (vegetable)

Fuel (diesel)

Fuel (kerosene)

Fuel (petrol-gasoline)

Maize flour

Rice

Wheat flour

Meat (beef)

Meat (goat)

Milk (UHT)

Sugar

Cooking fat

Bananas

Kale

Onions (red)

Tomatoes

Potatoes (Irish, red)

Beans (kidney)

Beans (rosecoco)

Beans (yellow)

Cabbage

Onions (dry)

Spinach

Potatoes (Irish, white)

Rice (aromatic)

Sorghum (red)

Beans (dolichos)

Cowpeas

Cowpea leaves

Maize (white, dry)

Beans (mung)

Millet (finger)

Rice (imported, Pakistan)

Fish (omena, dry)

Sorghum (white)

Salt

Meat (camel)

Milk (camel, fresh)

Milk (cow, fresh)


In [21]:
# do a quality mapping on commodity column
import re
def commodity(text):
    text = text.lower()
    if re.match(r'^ma', text):
        text = 'Maize'
    if re.match(r'^b', text):
        text = 'Beans'
    if re.match(r'^p', text):
        text = 'Potatoes'
    if re.match(r'^mi', text):
        text = 'Milk'
    if re.match(r'^fu', text):
        text = 'Fuel'
    if re.match(r'^mea', text):
        text = 'Meat'
    if re.match(r'^oi', text):
        text = 'Oil'
    if re.match(r'^on', text):
        text = 'Onion'
    if re.match(r'^ri', text):
        text = 'Rice'
    if re.match(r'^so', text):
        text = 'Sorghum'
    if re.match(r'^cow', text):
        text = 'Cowpeas'
    if re.match(r'^mil', text):
        text = 'Millet'
    if re.match(r'^fis', text):
        text = 'Fish'
    return text
        
df1['commodity'] = df1['commodity'].apply(commodity)
print('Unique values in Commodity column: ')
print('-------------------------------')
print(*df1['commodity'].unique(), sep='\n')
print('-------------------------------')

Unique values in Commodity column: 

-------------------------------

#item+name

Maize

Beans

Potatoes

Sorghum

Milk

Oil

Fuel

Rice

wheat flour

Meat

sugar

cooking fat

kale

Onion

tomatoes

cabbage

spinach

Cowpeas

Fish

salt

-------------------------------


In [22]:
# check unit column 
print(*df1['unit'].unique(), sep='\n')

#item+unit

KG

90 KG

50 KG

400 G

500 ML

L

200 ML

Unit

Bunch

126 KG

13 KG

64 KG

200 G

Head


In [23]:
# do a quality mapping on unit column
import re
def unit(text):
    text = text.lower()
    if text == 'bunch' or text == 'head':
        return text
    text = re.sub(r'(\d+\s*kg|\d+\s*g)', '1 KG', text)
    text = re.sub(r'(\d+\s*ml|\d+\s*l)', '1 L', text)
    text = re.sub(r'(?<!\d)l(?!\w)', '1 L', text)
    return text



df1['unit'] = df1['unit'].apply(unit)
print('Unique values in Unit column: ')
print('-------------------------------')
print(*df1['unit'].unique(), sep='\n')
print('-------------------------------')

Unique values in Unit column: 

-------------------------------

#item+unit

kg

1 KG

1 L

unit

bunch

head

-------------------------------


In [24]:
# df1.columns

In [25]:
# confirm unique values of our columns
excluded_cols = ['Province', 'County', 'market', 'price', 'usdprice', 'latitude', 'longitude']
for col in df1.columns:
    if col not in excluded_cols:
        unique_values = "\n".join(df1[col].unique().astype(str))
        print(f'{col} : {df1[col].nunique()} : {unique_values}\n')
        print('------------------------------------------')

category : 9 : #item+type

cereals and tubers

pulses and nuts

milk and dairy

oil and fats

non-food

meat, fish and eggs

miscellaneous food

vegetables and fruits



------------------------------------------

commodity : 21 : #item+name

Maize

Beans

Potatoes

Sorghum

Milk

Oil

Fuel

Rice

wheat flour

Meat

sugar

cooking fat

kale

Onion

tomatoes

cabbage

spinach

Cowpeas

Fish

salt



------------------------------------------

unit : 7 : #item+unit

kg

1 KG

1 L

unit

bunch

head



------------------------------------------

priceflag : 4 : #item+price+flag

actual

aggregate

forecast



------------------------------------------

pricetype : 3 : #item+price+type

Wholesale

Retail



------------------------------------------

currency : 2 : #currency

KES



------------------------------------------


In [26]:
# df1.dtypes

# 5. Feature Engineering
- Here we will pick our essential features for training the model

In [27]:
# let us see our columns
df1.columns

Index(['Province', 'County', 'market', 'latitude', 'longitude', 'category',
       'commodity', 'unit', 'priceflag', 'pricetype', 'currency', 'price',
       'usdprice'],
      dtype='object')

In [28]:
# create a copy of the df1 column
df2 = df1.copy()

In [29]:
# drop unnecessary columns
df2.drop(columns=['usdprice', 'currency'], axis=1, inplace=True)

In [30]:
# new columns to fit our model with
df2.columns

Index(['Province', 'County', 'market', 'latitude', 'longitude', 'category',
       'commodity', 'unit', 'priceflag', 'pricetype', 'price'],
      dtype='object')

In [31]:
df2.dtypes

Province      object
County        object
market        object
latitude     float64
longitude    float64
category      object
commodity     object
unit          object
priceflag     object
pricetype     object
price        float64
dtype: object

In [32]:
# encode the choosen columns
from sklearn.preprocessing import LabelEncoder

cat_cols = [column for column in df2.columns if df2[column].dtype == 'object']

# binary cols are the columns with only two unique values
binary_cols = [column for column in cat_cols if df2[column].nunique() == 3]
print(*binary_cols)

pricetype


In [33]:
df2 = pd.get_dummies(df2, columns=binary_cols, drop_first=True)
dummied_cols = [column for column in df2.columns if column not in cat_cols and column not in binary_cols]
df2.head()

,Province,County,market,latitude,longitude,category,commodity,unit,priceflag,price,pricetype_Retail,pricetype_Wholesale
date,,,,,,,,,,,,
#date,#adm1+name,#adm2+name,#loc+market+name,0.00,0.000000,#item+type,#item+name,#item+unit,#item+price+flag,0.00,0,0
2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize,kg,actual,16.13,0,1
2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,cereals and tubers,Maize,1 KG,actual,1480.00,0,1
2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans,kg,actual,33.63,0,1
2006-01-15,Coast,Mombasa,Mombasa,-4.05,39.666667,pulses and nuts,Beans,1 KG,actual,3246.00,0,1


In [34]:
# Encode the categorical variables to numeric values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_cols = [column for column in df2.columns if df2[column].dtype == 'object']

# label encode the categorical columns
for column in cat_cols:
    df2[column] = le.fit_transform(df2[column])

df2.head()

,Province,County,market,latitude,longitude,category,commodity,unit,priceflag,price,pricetype_Retail,pricetype_Wholesale
date,,,,,,,,,,,,
#date,0,0,0,0.00,0.000000,0,0,0,0,0.00,0,0
2006-01-15,2,14,50,-4.05,39.666667,1,5,5,1,16.13,0,1
2006-01-15,2,14,50,-4.05,39.666667,1,5,1,1,1480.00,0,1
2006-01-15,2,14,50,-4.05,39.666667,7,1,5,1,33.63,0,1
2006-01-15,2,14,50,-4.05,39.666667,7,1,1,1,3246.00,0,1


In [36]:
# split into train and test split
X = df2.drop('price', axis=1)
y = df2['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11603, 11), (2901, 11), (11603,), (2901,))

In [47]:
# Initialize the model
from sklearn.metrics import r2_score
cat = CatBoostRegressor(loss_function='RMSE', n_estimators=100, learning_rate=0.05, max_depth=5)
xgb = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.05, max_depth=5)
linear = LinearRegression()
rf = RandomForestRegressor()
models= [cat,linear, rf,xgb]

# Fit the model to the training data
for model in models:
    model = model.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    #metric
    mse = r2_score(y_test, y_pred) * 100
    print(f'error for {model} is: {mse}')

0:	learn: 2745.3808962	total: 3.32ms	remaining: 329ms

1:	learn: 2638.8287645	total: 6ms	remaining: 294ms

2:	learn: 2538.7706867	total: 7.97ms	remaining: 258ms

3:	learn: 2449.1309117	total: 10.4ms	remaining: 249ms

4:	learn: 2363.8934738	total: 12.6ms	remaining: 239ms

5:	learn: 2284.8601480	total: 14.5ms	remaining: 228ms

6:	learn: 2211.9475751	total: 16.3ms	remaining: 217ms

7:	learn: 2142.2277855	total: 18.4ms	remaining: 212ms

8:	learn: 2073.5772624	total: 20.1ms	remaining: 203ms

9:	learn: 2010.5891108	total: 22ms	remaining: 198ms

10:	learn: 1954.2749022	total: 23.2ms	remaining: 188ms

11:	learn: 1898.1363973	total: 25ms	remaining: 183ms

12:	learn: 1845.4823356	total: 26.9ms	remaining: 180ms

13:	learn: 1799.0337097	total: 28.8ms	remaining: 177ms

14:	learn: 1756.8080484	total: 30.2ms	remaining: 171ms

15:	learn: 1713.6163492	total: 31.7ms	remaining: 166ms

16:	learn: 1674.2762593	total: 33.5ms	remaining: 163ms

17:	learn: 1636.1482103	total: 35.1ms	remaining: 160ms

18:	learn

# CONCLUSION!
- From the trained models above,.. RandomForestRegressor, XGBRegressor and CatBoostRegressor are doing exemplary good. RandomForestRegressor tends to be best... tuning both XGBRegressor and CatBoostRegressor can be good

In [38]:
# testing random forest with random data
rf = RandomForestRegressor()
rf = rf.fit(X_train, y_train)
rf

RandomForestRegressor()

In [41]:
# df.head()

In [42]:
df2.head()

,Province,County,market,latitude,longitude,category,commodity,unit,priceflag,price,pricetype_Retail,pricetype_Wholesale
date,,,,,,,,,,,,
#date,0,0,0,0.00,0.000000,0,0,0,0,0.00,0,0
2006-01-15,2,14,50,-4.05,39.666667,1,5,5,1,16.13,0,1
2006-01-15,2,14,50,-4.05,39.666667,1,5,1,1,1480.00,0,1
2006-01-15,2,14,50,-4.05,39.666667,7,1,5,1,33.63,0,1
2006-01-15,2,14,50,-4.05,39.666667,7,1,1,1,3246.00,0,1
